In [1]:
import torch
import torch.nn as nn
import torch.utils.checkpoint as checkpoint
from timm.models.layers import DropPath, to_2tuple, trunc_normal_

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


In [3]:
def window_partition(x, window_size):
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C)
    return windows

def window_reverse(windows, window_size, H, W):
    B = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)
    return x


In [4]:
class WindowAttention(nn.Module):
    """ 윈도우 기반 멀티헤드 셀프 어텐션 (W-MSA) 모듈. 이는 시프트된 윈도우를 지원함.
    
    Args:
        dim (int): 입력 채널 수.
        window_size (tuple[int]): 윈도우의 높이와 너비.
        num_heads (int): 어텐션 헤드의 수.
        qkv_bias (bool, optional): 쿼리, 키, 밸류에 학습 가능한 바이어스를 추가할지 여부. 기본값: True
        qk_scale (float | None, optional): 헤드 차원 ** -0.5의 기본 qk 스케일을 오버라이드할 값. 기본값: None
        attn_drop (float, optional): 어텐션 가중치의 드롭아웃 비율. 기본값: 0.0
        proj_drop (float, optional): 출력의 드롭아웃 비율. 기본값: 0.0
    """
    def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.dim = dim
        self.window_size = window_size  # 윈도우의 높이와 너비 (Wh, Ww)
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        # 상대적 위치 바이어스 파라미터 정의
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads))

        # 각 토큰에 대한 상대적 위치 인덱스 계산
        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        coords = torch.stack(torch.meshgrid([coords_h, coords_w]))  # 2, Wh, Ww
        coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
        relative_coords[:, :, 0] += self.window_size[0] - 1  # 0부터 시작하도록 이동
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
        self.register_buffer("relative_position_index", relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        """ 순방향 전파.
        
        Args:
            x: 입력 특성 (num_windows*B, N, C)
            mask: 마스크 (0/-inf) (num_windows, Wh*Ww, Wh*Ww) 또는 None
        """
        B_, N, C = x.shape
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))

        relative_position_bias = self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()
        attn = attn + relative_position_bias.unsqueeze(0)

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


In [5]:
class SwinTransformerBlock(nn.Module):
    """ Swin Transformer 블록

    Args:
        dim (int): 입력 채널의 수
        input_resolution (tuple[int]): 입력 해상도 (높이, 너비)
        num_heads (int): 어텐션 헤드의 수
        window_size (int): 윈도우의 크기
        shift_size (int): 윈도우 시프트 크기
        mlp_ratio (float): MLP의 hidden dimension과 embedding dimension의 비율
        qkv_bias (bool, optional): QKV에 바이어스 추가 여부
        qk_scale (float | None, optional): QK 스케일링 인자 (기본값: head_dim ** -0.5)
        drop (float, optional): 드롭아웃 확률
        attn_drop (float, optional): 어텐션 드롭아웃 확률
        drop_path (float, optional): 드롭 패스 확률
        act_layer (nn.Module, optional): 활성화 함수 레이어
        norm_layer (nn.Module, optional): 정규화 레이어
    """
    def __init__(self, dim, input_resolution, num_heads, window_size=7, shift_size=0,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio

        if min(self.input_resolution) <= self.window_size:
            # 입력 해상도가 윈도우 크기보다 작거나 같으면 윈도우 크기 조정
            self.shift_size = 0
            self.window_size = min(self.input_resolution)

        assert 0 <= self.shift_size < self.window_size, "shift_size는 0과 window_size 사이의 값이어야 함"

        self.norm1 = norm_layer(dim)  # 첫 번째 정규화 레이어
        self.attn = WindowAttention(  # 윈도우 어텐션 레이어
            dim, window_size=to_2tuple(self.window_size), num_heads=num_heads,
            qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)

        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()  # 드롭 패스
        self.norm2 = norm_layer(dim)  # 두 번째 정규화 레이어
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)  # MLP 레이어

        if self.shift_size > 0:
            # 어텐션 마스크 계산
            H, W = self.input_resolution
            img_mask = torch.zeros((1, H, W, 1))  # 이미지 마스크 초기화
            h_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            w_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    cnt += 1

            mask_windows = window_partition(img_mask, self.window_size)  # 윈도우 분할
            mask_windows = mask_windows.view(-1, self.window_size * self.window_size)
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        else:
            attn_mask = None

        self.register_buffer("attn_mask", attn_mask)

    def forward(self, x):
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "입력 특징의 크기가 올바르지 않음"

        shortcut = x
        x = self.norm1(x)
        x = x.view(B, H, W, C)

        # 순환 이동
        if self.shift_size > 0:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
            # 윈도우 분할
            x_windows = window_partition(shifted_x, self.window_size)
        else:
            shifted_x = x
            x_windows = window_partition(shifted_x, self.window_size)

        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)

        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows, mask=self.attn_mask)

        # 윈도우 병합
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)

        # 순환 이동 복구
        if self.shift_size > 0:
            shifted_x = window_reverse(attn_windows, self.window_size, H, W)
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = window_reverse(attn_windows, self.window_size, H, W)

        x = x.view(B, H * W, C)

        # 잔차 연결 및 드롭 패스
        x = shortcut + self.drop_path(x)

        # MLP
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x


In [6]:
class PatchMerging(nn.Module):
    """ 패치 병합 레이어: 인접한 4개의 패치를 하나로 병합

    Args:
        input_resolution (tuple[int]): 입력 해상도 (높이, 너비)
        dim (int): 입력 채널 수
        norm_layer (nn.Module, optional): 정규화 레이어
    """
    def __init__(self, input_resolution, dim, norm_layer=nn.LayerNorm):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.reduction = nn.Linear(4 * dim, 2 * dim, bias=False)  # 차원 감소를 위한 선형 레이어
        self.norm = norm_layer(4 * dim)  # 정규화 레이어

    def forward(self, x):
        """
        x: (B, H*W, C) 형태의 텐서
        """
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "입력 특징의 크기가 올바르지 않음"
        assert H % 2 == 0 and W % 2 == 0, "H와 W는 2의 배수여야 함"

        # 패치를 재배열하여 4개의 그룹으로 나눔
        x = x.view(B, H, W, C)
        x0 = x[:, 0::2, 0::2, :]  # 첫 번째 그룹
        x1 = x[:, 1::2, 0::2, :]  # 두 번째 그룹
        x2 = x[:, 0::2, 1::2, :]  # 세 번째 그룹
        x3 = x[:, 1::2, 1::2, :]  # 네 번째 그룹
        x = torch.cat([x0, x1, x2, x3], -1)  # 4개 그룹 연결
        x = x.view(B, -1, 4 * C)  # B, H/2*W/2, 4*C

        x = self.norm(x)  # 정규화
        x = self.reduction(x)  # 차원 감소

        return x


In [7]:
class BasicLayer(nn.Module):
    """ Swin Transformer의 기본 레이어: 여러 Swin Transformer 블록과 패치 병합 레이어 포함

    Args:
        dim (int): 입력 채널 수
        input_resolution (tuple[int]): 입력 해상도 (높이, 너비)
        depth (int): 블록의 수
        num_heads (int): 어텐션 헤드의 수
        window_size (int): 윈도우의 크기
        mlp_ratio (float): MLP의 hidden dimension과 embedding dimension의 비율
        qkv_bias (bool, optional): QKV에 바이어스 추가 여부
        qk_scale (float | None, optional): QK 스케일링 인자
        drop (float, optional): 드롭아웃 확률
        attn_drop (float, optional): 어텐션 드롭아웃 확률
        drop_path (float | tuple[float], optional): 드롭 패스 확률
        norm_layer (nn.Module, optional): 정규화 레이어
        downsample (nn.Module | None, optional): 다운샘플링 레이어
        use_checkpoint (bool): 체크포인트 사용 여부
    """
    def __init__(self, dim, input_resolution, depth, num_heads, window_size,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., norm_layer=nn.LayerNorm, downsample=None, use_checkpoint=False):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.depth = depth
        self.use_checkpoint = use_checkpoint

        # Swin Transformer 블록들을 순차적으로 구성
        self.blocks = nn.ModuleList([
            SwinTransformerBlock(dim=dim, input_resolution=input_resolution,
                                 num_heads=num_heads, window_size=window_size,
                                 shift_size=0 if (i % 2 == 0) else window_size // 2,
                                 mlp_ratio=mlp_ratio,
                                 qkv_bias=qkv_bias, qk_scale=qk_scale,
                                 drop=drop, attn_drop=attn_drop,
                                 drop_path=drop_path[i] if isinstance(drop_path, list) else drop_path,
                                 norm_layer=norm_layer)
            for i in range(depth)])

        # 다운샘플링 레이어 (있을 경우)
        if downsample is not None:
            self.downsample = downsample(input_resolution, dim=dim, norm_layer=norm_layer)
        else:
            self.downsample = None

    def forward(self, x):
        for blk in self.blocks:
            if self.use_checkpoint:
                x = checkpoint.checkpoint(blk, x)
            else:
                x = blk(x)
        if self.downsample is not None:
            x = self.downsample(x)
        return x


In [8]:
class PatchEmbed(nn.Module):
    def __init__(self, img_size=224, patch_size=4, in_chans=3, embed_dim=96, norm_layer=None):
        super().__init__()
        img_size = to_2tuple(img_size)  # 여기를 수정합니다
        patch_size = to_2tuple(patch_size)  # 여기도 수정합니다
        self.img_size = img_size
        self.patch_size = patch_size
        self.patches_resolution = [img_size[0] // patch_size[0], img_size[1] // patch_size[1]]
        self.num_patches = self.patches_resolution[0] * self.patches_resolution[1]

        self.in_chans = in_chans
        self.embed_dim = embed_dim

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
        if norm_layer is not None:
            self.norm = norm_layer(embed_dim)
        else:
            self.norm = None

    def forward(self, x):
        B, C, H, W = x.shape
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"입력 이미지 크기 ({H}*{W})가 모델에 설정된 크기 ({self.img_size[0]}*{self.img_size[1]})와 다름"
        
        x = self.proj(x).flatten(2).transpose(1, 2)
        if self.norm is not None:
            x = self.norm(x)
        return x


In [11]:
class SwinTransformer(nn.Module):
    """ Swin Transformer 전체 모델

    Args:
        img_size (int): 입력 이미지 크기
        patch_size (int): 패치 크기
        in_chans (int): 입력 채널 수
        num_classes (int): 분류할 클래스 수
        embed_dim (int): 임베딩 차원
        depths (list[int]): 각 스테이지의 블록 수
        num_heads (list[int]): 각 스테이지의 헤드 수
        window_size (int): 윈도우 크기
        mlp_ratio (float): MLP 비율
        qkv_bias (bool): QKV에 바이어스 추가 여부
        qk_scale (float): QK 스케일링 인자
        drop_rate (float): 드롭아웃 확률
        attn_drop_rate (float): 어텐션 드롭아웃 확률
        drop_path_rate (float): 드롭 패스 확률
        norm_layer (nn.Module): 정규화 레이어
        ape (bool): 절대 위치 임베딩 사용 여부
        patch_norm (bool): 패치 정규화 사용 여부
        use_checkpoint (bool): 체크포인트 사용 여부
    """
    def __init__(self, img_size=224, patch_size=4, in_chans=3, num_classes=1000,
                 embed_dim=96, depths=[2, 2, 6, 2], num_heads=[3, 6, 12, 24],
                 window_size=7, mlp_ratio=4., qkv_bias=True, qk_scale=None,
                 drop_rate=0., attn_drop_rate=0., drop_path_rate=0.1,
                 norm_layer=nn.LayerNorm, ape=False, patch_norm=True,
                 use_checkpoint=False, **kwargs):
        super().__init__()

        self.num_classes = num_classes
        self.embed_dim = embed_dim
        self.ape = ape
        self.patch_norm = patch_norm

        # 패치 임베딩 레이어
        self.patch_embed = PatchEmbed(
            img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim,
            norm_layer=norm_layer if patch_norm else None)

        self.pos_drop = nn.Dropout(p=drop_rate)  # 위치 드롭아웃

        # 스테이지별 레이어 생성
        self.layers = nn.ModuleList()
        for i_layer in range(len(depths)):
            layer = BasicLayer(
                dim=int(embed_dim * 2 ** i_layer),
                input_resolution=(self.patch_embed.patches_resolution[0] // (2 ** i_layer),
                                  self.patch_embed.patches_resolution[1] // (2 ** i_layer)),
                depth=depths[i_layer],
                num_heads=num_heads[i_layer],
                window_size=window_size,
                mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate,
                drop_path=drop_path_rate,
                norm_layer=norm_layer,
                downsample=PatchMerging if i_layer < len(depths) - 1 else None,
                use_checkpoint=use_checkpoint)
            self.layers.append(layer)
            
        final_dim = embed_dim * 2 ** (len(depths) - 1)  # 최종 차원 계산
        self.norm = norm_layer(final_dim)  # 최종 출력 정규화 레이어
        self.head = nn.Linear(final_dim, num_classes) if num_classes > 0 else nn.Identity()  # 분류기 레이어

    def forward(self, x):
        x = self.patch_embed(x)
        x = self.pos_drop(x)

        for layer in self.layers:
            x = layer(x)

        x = self.norm(x)
        x = x.mean(dim=1)  # 글로벌 평균 풀링
        x = self.head(x)

        return x


In [12]:
# 모델 인스턴스 생성
model = SwinTransformer()
# 더미 입력 이미지 생성
dummy_img = torch.randn(1, 3, 224, 224)  # 배치 크기 1, 채널 3, 크기 224x224
# 모델을 통과시킴
output = model(dummy_img)
# 출력 텐서의 형태 출력
print(output.shape)


torch.Size([1, 1000])
